In [1]:
import pandas as pd
import numpy as np

In [2]:
seasons = pd.read_csv('csvs/seasons.csv')
metrics = pd.read_csv('csvs/metrics.csv')
team_games = pd.read_csv('csvs/team_games.csv')
team_seasons = pd.read_csv('csvs/team_seasons.csv')
team_colors = pd.read_csv('csvs/team_colors.csv')

In [7]:
metrics['stat_name']

0        tm_to_pct
1       opp_to_pct
2       tm_drb_pct
3       tm_orb_pct
4       tm_efg_pct
5      opp_efg_pct
6      poss_per_40
7       tm_net_eff
8       tm_off_eff
9       tm_def_eff
10     tm_ftm_rate
11    opp_ftm_rate
Name: stat_name, dtype: object

In [4]:
team_seasons_stats_melt = pd.melt(
  team_seasons[['season', 'tm_code', 'tm'] + metrics['stat_name'].tolist()],
  id_vars = ['season', 'tm_code', 'tm'],
  value_vars = metrics['stat_name'].tolist(),
  var_name = 'stat_name',
  value_name = 'raw_stat'
  )
team_seasons_stats_melt

,season,tm_code,tm,stat_name,raw_stat
0,2018,-1,NON-D1 TMS,tm_to_pct,24.109130
1,2018,2,Abilene Christian,tm_to_pct,17.747774
2,2018,5,Akron,tm_to_pct,17.159743
3,2018,6,Alabama A&M,tm_to_pct,22.327287
4,2018,7,Alabama St.,tm_to_pct,20.739845
...,...,...,...,...,...
21139,2014,19651,High Point,opp_ftm_rate,0.219593
21140,2014,26172,A&M-Corpus Christi,opp_ftm_rate,0.314847
21141,2014,28600,Lipscomb,opp_ftm_rate,0.312573
21142,2014,28755,FGCU,opp_ftm_rate,0.222460


In [9]:
season_stats = (team_seasons_stats_melt[['season', 'stat_name']].
  drop_duplicates().
  reset_index(drop = True)
  )
season_stats.head(2)

,season,stat_name
0,2018,tm_to_pct
1,2017,tm_to_pct


In [11]:
team_games['season_tm'] = (team_games['season'].map(str) + '_' + 
  team_games['tm_code'].map(str))

team_games['season_opp'] = (team_games['season'].map(str) + '_' + 
  team_games['opp_code'].map(str))

team_games.sample(2)

,season,game_date,game_id,tm_code,tm,tm_div,opp_code,opp,opp_div,is_home,...,tm_efg_pct,tm_to_pct,tm_orb_pct,tm_ftm_rate,opp_efg_pct,opp_to_pct,tm_drb_pct,opp_ftm_rate,season_tm,season_opp
48072,2014,2015-03-07,306-416-2015-03-07,416,Michigan St.,1.0,306,Indiana,1.0,0,...,49.21875,11.851852,34.883721,0.171875,54.807692,20.740741,78.125,0.288462,2014_416,2014_306
50711,2014,2015-02-07,255-749-2015-02-07,749,Wake Forest,1.0,255,Georgia Tech,1.0,0,...,35.15625,13.124316,27.272727,0.218750,53.389831,14.582574,70.000,0.169492,2014_749,2014_255


In [13]:
from sklearn.linear_model import Ridge

reg = Ridge(alpha=1,fit_intercept=True)

reg_results_collection = pd.DataFrame(columns = ['season', 'stat_name',
  'coef_name', 'ridge_reg_coef', 'ridge_reg_value'])

reg_results_collection

,season,stat_name,coef_name,ridge_reg_coef,ridge_reg_value


In [17]:
for index, row in season_stats.iterrows():
    this_season_game_stat = (team_games[team_games['season'] == row['season']]
    [['season_tm', 'tm_hca', 'season_opp', row['stat_name']]].
    reset_index()
    )

    this_season_game_dummy_vars = pd.get_dummies(
    this_season_game_stat[['season_tm', 'tm_hca', 'season_opp']]
    )
    break
this_season_game_stat

,index,season_tm,tm_hca,season_opp,tm_to_pct
0,0,2018_810,0,2018_794,15.372790
1,1,2018_794,0,2018_810,24.596464
2,2,2018_178,0,2018_632,17.175888
3,3,2018_632,0,2018_178,19.818332
4,4,2018_610,0,2018_627,19.226918
...,...,...,...,...,...
11425,11425,2018_108,1,2018_629,26.200873
11426,11426,2018_1014,1,2018_1320,11.594203
11427,11427,2018_1320,-1,2018_1014,15.942029
11428,11428,2018_473,-1,2018_101,25.039124
